# Prediction of leaderboard score (R)

In [6]:
score_data = read.csv('../input/scores.csv')
score_data[with(score_data, order(leaderboard_score)), ]

,model,leaderboard_score,accuracy,logloss,AUC,f1,mu,std
18,bagged_nolearn,0.4313,0.7813,7.5554,0.5857,0.3152,NA,NA
19,LogisticRegression,0.4411,0.7708,7.9152,0.554,0.2235,0.7601,0.0013
21,bagged_logit,0.4442,0.783,7.4954,0.5794,0.2938,0.76,0.0015
3,GradientBoostingClassifier,0.4452,0.7934,7.1356,0.6309,0.4251,0.7538,0.0047
20,LogisticRegressionCV,0.4457,0.783,7.4954,0.5794,0.2938,0.7602,0.0012
16,bagged_gbc,0.4527,0.7899,7.2556,0.6137,0.3858,0.7573,0.0037
1,nolearn,0.4566,0.8056,6.7159,0.6711,0.5044,NA,NA
4,ExtraTreesClassifier,0.4729,0.776,7.7353,0.5996,0.3582,0.7526,0.0061
5,XGBClassifier,0.4851,0.7743,7.7953,0.6034,0.3689,0.7474,0.0065
13,BaggingClassifier,0.4885,0.7743,7.7952,0.5985,0.3564,0.7523,0.0051


# missing mu & std get the column means

In [7]:
# for(i in 7:8){
#   score_data[is.na(score_data[,i]), i] <- mean(score_data[,i], na.rm = TRUE)
# }
# score_data

# Model using all variables

In [8]:
lm.fit = lm(leaderboard_score ~ accuracy + logloss + AUC + f1 + mu + std, 
            data      = score_data, 
            na.action = na.omit)

slm.fit = step(lm.fit, direction = "both")
summary(slm.fit)

Start:  AIC=-40.65
leaderboard_score ~ accuracy + logloss + AUC + f1 + mu + std

           Df Sum of Sq     RSS     AIC
- f1        1   0.00546 0.28785 -42.381
- AUC       1   0.01931 0.30169 -41.724
<none>                  0.28238 -40.650
- accuracy  1   0.16346 0.44585 -36.256
- logloss   1   0.16586 0.44824 -36.181
- std       1   0.17999 0.46238 -35.746
- mu        1   0.42767 0.71006 -29.741

Step:  AIC=-42.38
leaderboard_score ~ accuracy + logloss + AUC + mu + std

           Df Sum of Sq     RSS     AIC
<none>                  0.28785 -42.381
+ f1        1   0.00546 0.28238 -40.650
- accuracy  1   0.17025 0.45809 -37.876
- logloss   1   0.17241 0.46026 -37.810
- std       1   0.21652 0.50436 -36.529
- AUC       1   0.42327 0.71112 -31.720
- mu        1   0.49212 0.77996 -30.426



Call:
lm(formula = leaderboard_score ~ accuracy + logloss + AUC + mu + 
    std, data = score_data, na.action = na.omit)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.275512 -0.029650 -0.001879  0.084887  0.265253 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)   
(Intercept) -6127.1978  2808.5613  -2.182  0.06071 . 
accuracy     6112.6970  2810.1455   2.175  0.06132 . 
logloss       177.9199    81.2793   2.189  0.06002 . 
AUC             9.6952     2.8267   3.430  0.00896 **
mu              2.9213     0.7899   3.698  0.00606 **
std           -11.5539     4.7100  -2.453  0.03975 * 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.1897 on 8 degrees of freedom
  (8 observations deleted due to missingness)
Multiple R-squared:  0.8488,	Adjusted R-squared:  0.7543 
F-statistic: 8.982 on 5 and 8 DF,  p-value: 0.003888


# Remove mu & std (b/c of missing values)

In [9]:
# lm.fit = lm(leaderboard_score ~ accuracy + logloss + AUC + f1, 
#             data      = score_data, 
#             na.action = na.omit)

# slm.fit = step(lm.fit, direction = "both")
# summary(slm.fit)

In [10]:
score_data = score_data[with(score_data, order(leaderboard_score)), ]

library(knitr)
foo = kable(score_data, format = "markdown", digits = 4)
foof = ''
for (i in 1:length(foo)) {
    subs = substr(foo[i],5,52)
    foof = cat(foof,cat(subs,'\n'))
}
foof

|model                      | leaderboard_score| 
|:--------------------------|-----------------:| 
|bagged_nolearn             |            0.4313| 
|LogisticRegression         |            0.4411| 
|bagged_logit               |            0.4442| 
|GradientBoostingClassifier |            0.4452| 
|LogisticRegressionCV       |            0.4457| 
|bagged_gbc                 |            0.4527| 
|nolearn                    |            0.4566| 
|ExtraTreesClassifier       |            0.4729| 
|XGBClassifier              |            0.4851| 
|BaggingClassifier          |            0.4885| 
|ensemble of averages       |            0.4896| 
|SVC                        |            0.5336| 
|SGDClassifier              |            0.5670| 
|cosine_similarity          |            0.5732| 
|KMeans                     |            0.6289| 
|AdaBoostClassifier         |            0.6642| 
|KNeighborsClassifier       |            1.1870| 
|RandomForestClassifier     |            1.7907| 


NULL